In [ ]:
import pandas as pd

In [ ]:
latent = pd.read_table('./data/sts/latent_features.txt', sep='\s', header=None, engine='python')
smiles = pd.read_table('./data/sts/smiles_all.txt',sep='\n', header=None)
labels = pd.read_csv('./data/sts/sts_real.csv')

In [ ]:
label = labels[['SMILES', 'Smell Percepts']]
label

In [ ]:
for index in label.index:
    if 'fruity' in label.loc[index, 'Smell Percepts']:
        label.loc[index, 'fruity'] = int(1)
    else:
        label.loc[index, 'fruity'] = int(0)

In [ ]:
label_real = label.sort_values('fruity', ascending=False).drop_duplicates('SMILES').sort_index()
label_real = label_real.reset_index().drop(['index', 'Smell Percepts'], axis=1)
label_real

In [ ]:
# 以下为准备mcgan数据
smiles.columns = ['SMILES']
smiles

In [ ]:
label = pd.merge(smiles, labels, on='SMILES', how='left')
label = label[['SMILES', 'Smell Percepts']]
label

In [ ]:
# 遍历每行看是否有fruity标签
for index in label.index:
    if 'fruity' in label.loc[index]['Smell Percepts']:
        label.loc[index, 'fruity'] = int(1)
    else:
        label.loc[index, 'fruity'] = int(0)
label

In [ ]:
label_real = label.sort_values('fruity', ascending=False).drop_duplicates('SMILES').sort_index()
label_real = label_real.reset_index()
label_real

In [ ]:
data_all = pd.concat([smiles, latent, label_real['fruity']], axis=1)
col = ['smiles']
name = 'jtvae_ld_'
for i in range(56):
    col.append(name + str(i))
col.append('label')
data_all.columns = col
data_all

In [ ]:
# A为non-fruity，B为fruity
data_a = data_all.iloc[~(data_all['label'].values == 1)].drop('label', axis=1)
data_b = data_all.iloc[(data_all['label'].values == 1)].drop('label', axis=1)

In [ ]:
test_size_a = 700
test_size_b = 500

data_a_test = data_a.sample(test_size_a, axis=0)
data_b_test = data_b.sample(test_size_b, axis=0)

test_ind_a = data_a_test.index.to_list()
test_ind_b = data_b_test.index.to_list()
data_a_train = data_a.drop(test_ind_a, axis=0)
data_b_train = data_b.drop(test_ind_b, axis=0)

In [ ]:
label_real.to_csv('./data/sts/sts_train.csv', index=None)
data_a_train.to_csv('./data/sts/X_JTVAE_gs_train_A.csv', index=None)
data_b_train.to_csv('./data/sts/X_JTVAE_gs_train_B.csv', index=None)
data_a_test.to_csv('./data/sts/X_JTVAE_gs_test_A.csv', index=None)
data_b_test.to_csv('./data/sts/X_JTVAE_gs_test_B.csv', index=None)